# Predicting Water Pump Failures in Tanzania
## PHASE THREE PROJECT 

## Author
## Austine Otieno
## 1. Introduction
This project aims to leverage data science and machine learning to predict the operational status of water wells in Tanzania. Using historical data provided by "DrivenData", the goal is to build a robust classification model that can identify whether a well is:
 - Functional
 - Functional needs repair
 -  Non-functional

 #### 1.1 PROJECT OVERVIEW
####     1.1.1 BUSINESS UNDERSTANDING 
####        1.1.2 Business Problem
The Government of Tanzania and NGOs are working to ensure the availability of clean and functional water sources across the country. Many wells exist, but their operational status varies — some are functional, others need repair, and some are completely non-functional. Repairing and maintaining these water points is costly, so prioritization is crucial.


#### 1.1.3 Key Questions
* What features most strongly predict a well’s functionality?
* Can we identify vulnerable regions or technologies?
* How well does our model generalize to unseen data?

### 2) DATA UNDERSTANDING 
### 2.1 DATA PREPROCESSING
### 2.1.1 The Data 
I sourced my Data from Kaggle -DrivenData- and the dataset includes:

* train_set_values.csv — Features describing -59,000 water wells.

* train_set_labels.csv — Target labels for those wells.

* test_set_values.csv — Data to predict on for the final submission.

### 2.1.2 The data structure consist of Over 40 features including:

* Location data: latitude, longitude, region

* Construction info: installer, construction year

* Management info: funder, management type

* Technical info: pump type, water quality, extraction type

* Target variable: status_group (3 classes)


2.1.3 DATA PREPARATION
This entails;

-Importing necessary libraries

-Loading and Accessing of the data set

-Merging: Combined train_set_values and train_set_labels using id.

-Data Cleaning and preparation which involves:Handled missing values and standardised text fields.

-Encoding:

* Used 'LabelEncoder' for categorical variables.

* Mapped 'status_group' to numeric classes for modeling.

### A) Importing necessary libraries

In [7]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### B) Load The Data Set 

In [9]:
X_train = pd.read_csv("Training_set_values.csv")
y_train = pd.read_csv("Trainig_test_labels.csv")
X_test = pd.read_csv("test_set_values.csv") 

print(X_train.shape, y_train.shape, X_test.shape)

(59400, 40) (59400, 2) (14850, 40)


### C) General inspectin of the data set

In [11]:
print(X_train.info()) # Displays the data types of each column. 
print(X_train.describe(include='all').T) # Shows descriptive statistics for all columns (including categorical columns) 
print(y_train['status_group'].value_counts()) # Counts how many samples belong to each class in the target column (status_group)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55763 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59398 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

* From the above we are able to see the data types of each column, their descriptive statistics and number of samples belonging to each class in the target column (status_group)

### 3: Handle Missing Data 
* 3.1 Drop columns with too many missing values

In [14]:
# Check missing values
missing = X_train.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])


scheme_name          28810
scheme_management     3878
installer             3655
funder                3637
public_meeting        3334
permit                3056
subvillage             371
wpt_name                 2
dtype: int64


* 3.2  Fill numeric columns with "median" and Fill categorical columns with "unknown" for both X_train and X_test.

In [16]:
       #(X_train)
# Fill missing numerical values
X_train['construction_year'].replace(0, pd.NA, inplace=True)
X_train['construction_year'].fillna(X_train['construction_year'].median(), inplace=True)

# Fill categorical nulls with 'unknown'
cat_cols = X_train.select_dtypes(include='object').columns
X_train[cat_cols] = X_train[cat_cols].fillna("unknown")

X_train[cat_cols]

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,unknown,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,2013-05-03,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,2011-05-07,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,2011-04-11,unknown,unknown,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala,True,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,2011-03-08,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [17]:
       # (X_test)
# Fill missing numerical values 
X_test['construction_year'].replace(0, pd.NA, inplace=True)
X_test['construction_year'].fillna(X_test['construction_year'].median(), inplace=True)

# Fill categorical nulls with 'unknown'
cat_cols = X_test.select_dtypes(include='object').columns
X_test[cat_cols] = X_train[cat_cols].fillna("unknown") 

X_test[cat_cols]


,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,unknown,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,2012-11-09,District Council,District council,Mwanzelwa,Internal,Lutona,Tabora,Uyui,Loya,True,...,never pay,milky,milky,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14846,2012-10-09,Dwsp,DWE,Shuleni,Internal,Ibadakuli Kati,Shinyanga,Shinyanga Urban,Ibadakuli,False,...,never pay,salty,salty,insufficient,insufficient,hand dtw,borehole,groundwater,hand pump,hand pump
14847,2011-02-21,Woyege,WOYEGE,Woyege,Wami / Ruvu,Kimbangulile,Pwani,Mkuranga,Tambani,True,...,per bucket,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14848,2013-10-03,Germany Republi,CES,Kwa Amphales Masaa,Pangani,Saa,Kilimanjaro,Hai,Machame Kaskazini,True,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


### 4: Feature Engineering 

* Convert construction_year to pump_age

In [19]:
import pandas as pd 
# Reload the data set to retrieve "construction_year" column 

X_train = pd.read_csv("Training_set_values.csv")
X_test = pd.read_csv("Test_set_values.csv")

# Treat invalid years (0) as missing
X_train['construction_year'].replace(0, pd.NA, inplace=True)
X_test['construction_year'].replace(0, pd.NA, inplace=True)

# Fill in missing years with a reasonable estimate
X_train['construction_year'].fillna(X_train['construction_year'].median(), inplace=True)
X_test['construction_year'].fillna(X_test['construction_year'].median(), inplace=True)


In [20]:
# create pump_age
X_train['pump_age'] = 2025 - X_train['construction_year']
X_test['pump_age'] = 2025 - X_test['construction_year'] 



X_train['pump_age']
X_test['pump_age'] 

0        13.0
1        25.0
2        15.0
3        38.0
4        25.0
         ... 
14845    37.0
14846    31.0
14847    15.0
14848    16.0
14849    17.0
Name: pump_age, Length: 14850, dtype: float64

*  Limit Cardinality Before Encoding by Combining rare categories since my model is trying to handle a very large number of features (47,520) and can't fit the data in memory (7.59 GB for one array).

In [22]:
# Step 1: Combine rare categories in categorical features (threshold: <1% frequency)
def combine_rare_categories(df, threshold=0.01):
    for col in df.select_dtypes(include='object').columns:
        freq = df[col].value_counts(normalize=True)
        rare_labels = freq[freq < threshold].index
        df[col] = df[col].replace(rare_labels, 'other')
    return df

# Apply to both train and test
X_train = combine_rare_categories(X_train)
X_test = combine_rare_categories(X_test)

# Step 2: Concatenate and one-hot encode (now with fewer categories)
combined = pd.concat([X_train, X_test], axis=0)
combined = pd.get_dummies(combined)

# Step 3: Split back
X_train_cleaned = combined.iloc[:X_train.shape[0], :]
X_test_cleaned = combined.iloc[X_train.shape[0]:, :]


### 4.1: Encode Categorical Features 

In [24]:
# Combine train and test to encode consistently
combined = pd.concat([X_train, X_test], axis=0)
combined = pd.get_dummies(combined)

# Split back
X_train_cleaned = combined.iloc[:X_train.shape[0], :]
X_test_cleaned = combined.iloc[X_train.shape[0]:, :]

X_train_cleaned
X_test_cleaned

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_communal standpipe,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,50785,0.0,1996,35.290799,-4.059696,0,21,3,321,2012.0,...,True,False,False,False,False,True,False,False,False,True
1,51630,0.0,1569,36.656709,-3.309214,0,2,2,300,2000.0,...,False,True,False,False,False,False,True,False,False,False
2,17168,0.0,1567,34.767863,-5.004344,0,13,2,500,2010.0,...,True,False,False,False,False,True,False,False,False,True
3,45559,0.0,267,38.058046,-9.418672,0,80,43,250,1987.0,...,False,False,False,False,False,True,False,False,False,True
4,49871,500.0,1260,35.006123,-10.950412,0,10,3,60,2000.0,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,34,38.852669,-6.582841,0,6,1,20,1988.0,...,True,True,False,False,False,False,True,False,False,False
14846,18990,1000.0,0,37.451633,-5.350428,0,4,7,2960,1994.0,...,False,False,False,True,False,False,False,True,False,False
14847,28749,0.0,1476,34.739804,-4.585587,0,13,2,200,2010.0,...,True,True,False,False,False,False,True,False,False,False
14848,33492,0.0,998,35.432732,-10.584159,0,10,2,150,2009.0,...,True,True,False,False,False,False,True,False,False,False


### This splits the combined DataFrame back into:
* X_train_cleaned: the top N rows (same number as in X_train)
* X_test_cleaned: the rest of the rows (same number as in X_test)

### 4.2: Encode Target

* returning a NumPy array of encoded labels.

In [28]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train['status_group'])

y_train_encoded

array([0, 0, 0, ..., 0, 0, 0])

 * data is now cleaned, encoded, and ready for modeling.

# 5. Modeling 

## 5.1 Split Training Data for Validation
*  split the training set to evaluate models during development.

In [32]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_cleaned, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded
)
 

## 5.2 Baseline Model 
### 5.2.1 (Logistic Regression) 

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Train the model 
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_tr, y_tr)

# Predict
y_pred = log_reg.predict(X_val)

# Evaluate
print("Logistic Regression Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))


Logistic Regression Accuracy: 0.5941077441077441
              precision    recall  f1-score   support

           0       0.59      0.89      0.71      6452
           1       0.00      0.00      0.00       863
           2       0.60      0.29      0.39      4565

    accuracy                           0.59     11880
   macro avg       0.40      0.39      0.37     11880
weighted avg       0.55      0.59      0.54     11880



## Interpretation of 5.2.1 
### Class Imbalance:

* Class 0 has a lot more samples than class 1 → the model learns to predict class 0 often.

* That’s why you see very high recall for class 0 (0.89), but 0 for class 1.

### Poor Generalization to Minority Classes:

* Class 1 is ignored: precision, recall, and F1-score are all 0.00 → this means the model never predicted class 1 at all.

* Class 2 is partially recognized, but only a few actual class 2 samples are correctly predicted (low recall).

### Linear Model Limitation:

* Logistic regression is a linear model. If the decision boundaries are non-linear, it will struggle to classify correctly — especially for underrepresented classes.



### 5.2.2  (Random Forest) 

In [42]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_tr, y_tr)
y_pred_rf = rf.predict(X_val)

print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))


Random Forest Accuracy: 0.8085858585858586
              precision    recall  f1-score   support

           0       0.81      0.90      0.85      6452
           1       0.54      0.31      0.39       863
           2       0.84      0.78      0.81      4565

    accuracy                           0.81     11880
   macro avg       0.73      0.66      0.68     11880
weighted avg       0.80      0.81      0.80     11880



### Interpretation of 5.2.2 Above 
* Overall accuracy is up to 80.8%—a big jump from 59%.

* Classes 0 and 2 are being predicted quite well with good precision and recall.

* Random Forest is clearly capturing more complex relationships in the data.

### 5.2.3 (SMOTE)

In [46]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


In [48]:
# Split the data
X_tr, X_val, y_tr, y_val = train_test_split(X_train_cleaned, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded)

# Apply SMOTE to training set 
smote = SMOTE(random_state=42)
X_tr_balanced, y_tr_balanced = smote.fit_resample(X_tr, y_tr)


In [50]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_tr_balanced, y_tr_balanced) 

# Evaluate
y_pred = rf.predict(X_val)
print("Random Forest Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Random Forest Accuracy: 0.8068181818181818
              precision    recall  f1-score   support

           0       0.81      0.89      0.85      6452
           1       0.52      0.32      0.40       863
           2       0.84      0.78      0.81      4565

    accuracy                           0.81     11880
   macro avg       0.72      0.66      0.69     11880
weighted avg       0.80      0.81      0.80     11880



## Intrepetation 
### Class 1 ("functional needs repair") remains the toughest:

* Only 32% of class 1 cases were correctly identified.

* Precision at 52% means many predictions for class 1 were incorrect.

## 5.3 Hyperparameters 

In [54]:
# Introduce new library 
from sklearn.model_selection import GridSearchCV

In [56]:
# Define parameter grid 
param_grid = {
    'n_estimators': [100, 150],       # number of trees
    'max_depth': [10, None],          # depth of each tree
    'min_samples_split': [2, 5,],     # min samples to split a node
    'min_samples_leaf': [2, 5],       # min samples at leaf node
     
}


In [58]:
#  Initialize the mode 
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV 
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,               # 3-fold cross-validation
    n_jobs=-1,          # use all CPU cores
    verbose=2,
    scoring='f1_macro'  # for imbalanced multiclass
)
grid_search

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, None], 'min_samples_leaf': [2, 5],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 150]},
             scoring='f1_macro', verbose=2)

In [60]:
# Fit to the SMOTE-balanced data 
grid_search.fit(X_tr, y_tr)

# Best parameters 
print("Best parameters:", grid_search.best_params_)
best_rf = grid_search.best_estimator_


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150}


In [66]:
# Evaluate 
y_pred = best_rf.predict(X_val)

from sklearn.metrics import classification_report, accuracy_score
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Accuracy: 0.8084175084175084
              precision    recall  f1-score   support

           0       0.79      0.92      0.85      6452
           1       0.62      0.24      0.35       863
           2       0.85      0.76      0.81      4565

    accuracy                           0.81     11880
   macro avg       0.76      0.64      0.67     11880
weighted avg       0.80      0.81      0.80     11880



## Interpretation 0f 5.3 Above- 
### Best Model so far
* The model strongly predicts class 0 and class 2, which together make up the majority of the dataset.

* Class 1 (needs repair) still has low recall (24%), meaning many of these are misclassified as other classes.

* Precision for class 1 is higher than before (62%), showing the model is more cautious but better when it does predict it.

# 6. Findings
#### 6.1 Dataset Characteristics 
* The dataset was highly imbalanced: most examples belonged to the "functional" class.

* construction_year had missing or zero values, which required imputation.

* Many categorical variables with high cardinality (e.g. funder, installer).

#### 6.2 Model Performance 
##### Logistic Regression
* Accuracy: ~59%

* Major Weakness: Completely failed to predict class 1 (non functional needs repair).

* F1-scores: Very low for minority classes.

* Interpretation: Linear model could not capture complex relationships; suffered due to class imbalance.

##### Random Forest (Before SMOTE)
* Accuracy: ~80%

* Precision & Recall:

* functional: Good

* non functional: Good

* needs repair: Poor recall (~32%) but better than logistic regression

* - Interpretation: Captured non-linear relationships better; still struggled with the rarest class.

##### Random Forest (After SMOTE)
* Balanced class distribution during training.

* Improved recall for minority class (needs repair), though gains were modest.

* Slight decrease in precision for majority class, but better macro F1-score.

#### 6.3 Hyperparameters 
* The model strongly predicts class 0 and class 2, which together make up the majority of the dataset.

* Class 1 (needs repair) still has low recall (24%), meaning many of these are misclassified as other classes.

* Precision for class 1 is higher than before (62%), showing the model is more cautious but better when it does predict it.

#  7.Recommendations

Based on my findings, I suggest the following strategies

## I. Model Improvements
* Use Random Forest with SMOTE: Best balance between performance and interpretability.

* Use Class Weights: If avoiding SMOTE, set class_weight='balanced' in your model to give rare classes more importance.

## II. Feature Engineering
* Derived Feature: pump_age helped improve performance.

* Explore feature importance from Random Forest to remove noisy variables.

## III. Hyperparameter Tuning
* Use a smaller parameter grid for resource-constrained environments.

* Consider RandomizedSearchCV for broader coverage at lower cost.

## IV. Evaluation Metrics
* Use F1-macro or weighted score over simple accuracy due to class imbalance.

* Plot confusion matrix to monitor per-class predictions visually.

## V. Production Readiness
* Ensure missing value strategies and encoders are applied consistently between training and test sets.


# 8. Conclusion: Pump It Up – Water Table Prediction Project
* This project aimed to build a machine learning model to predict the functional status of water pumps in Tanzania. After thorough data preprocessing, encoding, balancing, and model experimentation, the Random Forest Classifier with SMOTE and tuned hyperparameters produced the best performance.

# END 